# Batch Activation Counting Tools

This notebook is to aid in determining optimal count times given a set of activations and experiments design parameters.
It reads from a formatted excel input to obtain the data for multiple foils and plots the count time versus the irradiation time.

This is a good starting point to scope out experimental parameters, but it will not develop a count plan.  For that purpose, refer to 

Import the following packages:

In [1]:
import openpyxl
import sys
import os

import numpy as np
import matplotlib.pyplot as plt

from pyne import data
from pyne import nucname
from scipy.integrate import quad
from math import exp,sqrt

sys.path.insert(0,os.path.abspath('/home/pyne-user/Dropbox/UCB/Computational_Tools/Scripts/Python/GeneralNuclear'))
from BasicNuclearCalcs import fractional_solid_angle, activity, production_decay
from Counting import volume_solid_angle, germanium_eff

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:8: QAWarning: pyne.data is not yet QA compliant.


## User Experimental Input

Specify the experimental parameters determining the reaction rate.  This assumes that the T<sub>1/2</sub> is >> 1 seconds, which should be true for all practical experiments.

### Counting Facility Variables
background = the background rate at the peak of interest in counts/s

det_r = radius of the detector in cm

det_foil_dist = the distance from the detector face to the foil in cm

sigma = the desired counting statistics level

trans_t= the time to transfer the foil to the counting facility following irradiation in sec

### File Name
fname = path of the file containing the foil and beam data

In [6]:
background=0.01
det_r=3.245
det2foil_dist=1
sigma=0.01
trans_t=0

fname='/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Simulated/Activation/Vault_ETA_IRDFF/Foils.xlsx'  

## Read in foil data from Excel workbook and calculate run parameters

### Beam Variables
src = the neutron source strength in in n/sec

### Foil Variables
rx_prod = name of the foil reaction product written in the format "XXAAA" - ex "U235" or "Rb86"

energy = the line of interest in keV

BR = the branching ratio of the line of interest

rx_rate = the reaction rate per source particle in units of reactions/cm<sup>3</sup>/src (this can be obtained from simulation or a simple calculation). It assumes that the natural abundance of the isotope is accounted for in this rate.  

foil_r = the foil radius in cm

foil_h = the foil height in cm

vol = the foil volume in cm<sup>3</sup>

In [10]:
# Define the activity integrand accounting for all of the efficiencies
def integrand(t):
     return activity(half_life,n0,t)*abs_eff*BR/100

# Load Excel file and read all rows and tabs
wb = openpyxl.load_workbook(fname, data_only=True)
for tab in wb.get_sheet_names():
    # Import sheet object
    sheet = wb.get_sheet_by_name(tab)

    # Read in data
    i=2   #Skip the header
    plot_data=[]
    plot_labels=[]
    while sheet.cell(row=i, column=1).value != None:
        rx=str(sheet.cell(row=i, column=1).value)
        rx_prod=str(sheet.cell(row=i, column=2).value)
        energy=float(sheet.cell(row=i, column=3).value)
        BR=float(sheet.cell(row=i, column=4).value)
        src=float(sheet.cell(row=i, column=5).value)
        rx_rate=float(sheet.cell(row=i, column=6).value)
        foil_r=float(sheet.cell(row=i, column=8).value)
        foil_h=float(sheet.cell(row=i, column=9).value)
        half_life=float(sheet.cell(row=i, column=14).value)
        
        print "Reading {}".format(rx_prod)
        
        # Calculate the foil volume
        vol=np.pi*foil_r**2*foil_h
        i+=1
        
        # Calculate the GCF
        gcf=volume_solid_angle(foil_r,det_r,det2foil_dist)
        
        # Calculate absolute efficiency
        rel_eff=germanium_eff(energy)
        abs_eff=rel_eff*(gcf/fractional_solid_angle(det_r,det2foil_dist))
        
        # Calculate the count time for a given irradiation time and store the data
        timeprs=[]
        for src_t in range(60,86400,120):    # Irradiation time in seconds
            # Calculate the initial number of atoms from the irradiation (assumes natural abundance is captured in rx_rate)
            n0=production_decay(half_life, 0.0, src_t, rx_rate, src, vol, trans_t)
            
            # Approximate the optimal foil counting time using an average count rate
            tf=1
            diff=1000
            try:
                while diff > 1:
                    prevt=tf
                    S = quad(integrand, 0, tf)[0]/tf
                    tf=((sqrt(S+background)+sqrt(background))**2/(sigma**2*S**2))/(1+1/sqrt((S+background)/background))  #Knoll eqn 3.54/55
                    diff=tf-prevt
            except ZeroDivisionError:
                tf=1E99
            timeprs.append([src_t,tf]) 
        
        # Append to the data arrays
        plot_data.append(timeprs)
        plot_labels.append(rx)

Reading Zr89
Reading Ni57
Reading Co58
Reading In115M
Reading In116M
Reading Au196
Reading Au198
Reading Mg27
Reading Na24


### Plots

Bringing it all together....

In [12]:
# Allow use of Tex sybols
plt.rc('text', usetex=True)

# Set up figure
fig = plt.figure()
#ax1 = fig.add_subplot(111)
ax1 = fig.add_axes([0.1, 0.1, 0.8, 0.85])

# Preset data set format scheme
s=10
linewidth=['1.5','2.5','3.5','4.5']
marker=['.','o','v','^','<','>','1','2','3','4','8','s','p','*','h','H','+','x','d','D']
linestyle=[':', '-.', '--', '-','-']
dashes=[[2, 2, 2, 2], [10, 5, 2, 5], [10, 5, 10, 5], [10, 2, 2, 2, 2, 2],
        [10, 0.1]]

# Set Line color cycle
ax1.set_color_cycle(['k', 'k', 'k', 'k', 'k', 'r', 'r', 'r','r', 'r', 'b', 'b', 'b', 'b', 'b', 'grey', 'grey', 'grey', 'grey', 'grey'])
#ax1.set_color_cycle(['k','b','g','r','c','m','y','w'])

# Set axes
ax1.axis([.1, 8600, .1, 50*2900])
ax1.set_xscale('log')
ax1.set_yscale('log')

# Set axes labels and plot title.
# ax1.set_title("Plot title...")    
ax1.set_xlabel('\\textbf{Irradiation Time [min]}', fontsize=18, weight="bold")
ax1.set_ylabel('\\textbf{Count Time [min]}', fontsize=18, weight="bold")
ax1.tick_params(axis='both', which='major', labelsize=18, width=2)
ax1.tick_params(axis='both', which='minor', width=2)

# Loop over the list of data
for i in range(len(plot_labels)):
    # Reset x and y data lists and index for lists
    x=[]
    y=[]

    # Build list of x, y coord pairs  
    for d in plot_data[i]:
        x.append(d[0]/60) 
        y.append(d[1]/60)

    # Add data set to line plot
    ax1.plot(x, y, linewidth=linewidth[i//5], linestyle=linestyle[i%5], 
                label=plot_labels[i], dashes=dashes[i%5]) 

    # Add and locate legend
    leg = ax1.legend()
    plt.legend(borderaxespad=0.75, loc=2, fontsize=16, handlelength=5, borderpad=0.5,\
                labelspacing=0.75, fancybox=True, framealpha=0.5);

plt.show()